## Video latency analysis
LE640 day4 and LE679 day2 sessions (from Figure S2A) are used

In [2]:
import os
from pathlib import Path
import numpy as np
import pandas as pd

from scipy import stats

import pingouin as pg

import matplotlib as mpl
import matplotlib.pyplot as plt  

import seaborn as sns

from datetime import date
import time

from joblib import Parallel, delayed

import cv2

In [4]:
mother_path = Path('D:/Multi-modal project/')
video_path = mother_path / 'videos' / 'DVR backup' / 'ephys'

In [18]:
video1 = 'C:/Users/HP/Desktop/video/u40200.avi'
video2 = 'C:/Users/HP/Desktop/video/u40100.avi'

In [30]:
def extract_frames(video_path):
    frames = []
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Couldn't open the video file")
        return None

    frame_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        resize_frame = cv2.resize(frame, (352,240))
        frames.append(resize_frame)
        
        frame_count += 1

    cap.release()
    print(f"{video_path} completed")
    print(f"Frames extracted: {frame_count}")
    
    return frames

In [31]:
# frame extraction
frames1 = extract_frames(str(video1))
frames2 = extract_frames(str(video2))

C:/Users/HP/Desktop/video/u40200.avi completed
Frames extracted: 37157
C:/Users/HP/Desktop/video/u40100.avi completed
Frames extracted: 37158


In [56]:
def combine_frames_with_timestamp(frame1, frame2, output_path, sample_rate):

    # Create result file
    result = []
    
    # Initialize variables
    timestamp = 0
    frame_count = 0
    
    for f1, f2 in zip(frame1, frame2):
        combined_frame = np.hstack((f1, f2))

        # Add timestamps as text to the combined frame
        text1 = f'Frame: {frame_count}, Timestamp: {timestamp:.1f} ms'
        cv2.putText(combined_frame, text1, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        output_filename = f"{output_path}/frame_{frame_count:05d}.jpg"
        cv2.imwrite(output_filename, combined_frame)

        # make result
        result.append({'frame': frame_count, 'timestamp': timestamp, 'event':0})
        
        timestamp += sample_rate
        frame_count += 1
        
    print(f"{frame_count+1} frames are combined")
    return pd.DataFrame(result)

In [58]:
# combining two videos
result = combine_frames_with_timestamp(frames1, frames2, 
                              'C:/Users/HP/Desktop/video/frames',
                             1000/30)

37158 frames are combined


In [59]:
result.to_csv('C:/Users/HP/Desktop/video/timestamp_data.csv')